In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv(r"D:\ost\OST-SM-Project\datasets\swat\normal\SWaT_Dataset_Normal_v0_1.csv")

                Timestamp    FIT101    LIT101  MV101  P101  P102    AIT201  \
0   22/12/2015 4:00:00 PM  2.470294  261.5804      2     2     1  244.3284   
1   22/12/2015 4:00:01 PM  2.457163  261.1879      2     2     1  244.3284   
2   22/12/2015 4:00:02 PM  2.439548  260.9131      2     2     1  244.3284   
3   22/12/2015 4:00:03 PM  2.428338  260.2850      2     2     1  244.3284   
4   22/12/2015 4:00:04 PM  2.424815  259.8925      2     2     1  244.4245   

    AIT202   AIT203    FIT201  ...  P501  P502    PIT501  PIT502    PIT503  \
0  8.19008  306.101  2.471278  ...     1     1  10.02948     0.0  4.277749   
1  8.19008  306.101  2.468587  ...     1     1  10.02948     0.0  4.277749   
2  8.19008  306.101  2.467305  ...     1     1  10.02948     0.0  4.277749   
3  8.19008  306.101  2.466536  ...     1     1  10.02948     0.0  4.277749   
4  8.19008  306.101  2.466536  ...     1     1  10.02948     0.0  4.277749   

     FIT601  P601  P602  P603  Normal_Attack  
0  0.000256    

In [ ]:
df['label'] = df.iloc[:, -1].apply(lambda x: 1 if x == 'Normal' else -1)

normal_df = df[df['label'] == 1].copy()

In [ ]:
from sklearn.preprocessing import StandardScaler
X_normal = normal_df.drop(columns=['Timestamp', 'Normal_Attack', 'label'], errors='ignore')

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_normal)

In [ ]:
df_attack = pd.read_csv(r"D:\ost\OST-SM-Project\datasets\swat\attack\SWaT_Dataset_Attack_v0_1.csv") 

df_attack['label'] = df_attack.iloc[:, -1].apply(lambda x: 1 if x == 'Normal' else -1)
df_attack.columns = [col.strip() for col in df_attack.columns]
df_attack['label'] = df_attack['Normal_Attack'].apply(lambda x: 1 if x == 'Normal' else -1)

X_val = df_attack.drop(columns=['Timestamp', 'Normal_Attack', 'label'], errors='ignore')

X_val = X_val[X_normal.columns] 
X_val_scaled = scaler.transform(X_val)

# True labels for evaluation
y_val = df_attack['label'].values

In [6]:
df_attack

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal_Attack,label
0,28/12/2015 10:00:00 AM,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,...,1,250.8652,1.649953,189.5988,0.000128,1,1,1,Normal,1
1,28/12/2015 10:00:01 AM,2.446274,522.8860,2,2,1,262.0161,8.396437,328.6337,2.445391,...,1,250.8652,1.649953,189.6789,0.000128,1,1,1,Normal,1
2,28/12/2015 10:00:02 AM,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,...,1,250.8812,1.649953,189.6789,0.000128,1,1,1,Normal,1
3,28/12/2015 10:00:03 AM,2.534350,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,...,1,250.8812,1.649953,189.6148,0.000128,1,1,1,Normal,1
4,28/12/2015 10:00:04 AM,2.569260,523.4748,2,2,1,262.0161,8.394514,328.6337,2.443085,...,1,250.8812,1.649953,189.5027,0.000128,1,1,1,Normal,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449914,2/1/2016 2:59:55 PM,2.559972,519.5495,2,2,1,168.0979,8.638683,301.9226,2.459488,...,1,251.1535,0.865024,189.0220,0.000000,1,1,1,Normal,1
449915,2/1/2016 2:59:56 PM,2.549082,520.4131,2,2,1,168.0979,8.638683,301.9226,2.459488,...,1,251.0734,0.865024,188.9259,0.000000,1,1,1,Normal,1
449916,2/1/2016 2:59:57 PM,2.531467,520.6878,2,2,1,168.0979,8.638683,301.9226,2.460129,...,1,251.0734,0.865024,188.9259,0.000000,1,1,1,Normal,1
449917,2/1/2016 2:59:58 PM,2.521218,520.7271,2,2,1,168.0979,8.638683,301.9226,2.460129,...,1,251.0734,0.865024,188.9259,0.000000,1,1,1,Normal,1


In [7]:
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import StandardScaler

# Parameter grid
nus = [0.001, 0.005, 0.01, 0.02]
gammas = ['scale', 'auto', 0.0001, 0.001, 0.01]

In [8]:
from sklearn.svm import OneClassSVM

best_score = 0
best_model = None
best_params = {}

for nu in nus:
    for gamma in gammas:
        print(f"Training OCSVM with nu={nu}, gamma={gamma}")
        model = OneClassSVM(kernel='rbf', nu=nu, gamma=gamma)
        model.fit(X_train_scaled)
        
        # Predict on attack set
        preds = model.predict(X_val_scaled)

        # Evaluate
        f1 = f1_score(y_val, preds, pos_label=-1)  # focus on anomaly class
        print(f"F1 Score: {f1:.4f}")

        if f1 > best_score:
            best_score = f1
            best_model = model
            best_params = {'nu': nu, 'gamma': gamma}

Training OCSVM with nu=0.001, gamma=scale
F1 Score: 0.2544
Training OCSVM with nu=0.001, gamma=auto
F1 Score: 0.2541
Training OCSVM with nu=0.001, gamma=0.0001
F1 Score: 0.2961
Training OCSVM with nu=0.001, gamma=0.001
F1 Score: 0.2855
Training OCSVM with nu=0.001, gamma=0.01
F1 Score: 0.2606
Training OCSVM with nu=0.005, gamma=scale
F1 Score: 0.2542
Training OCSVM with nu=0.005, gamma=auto
F1 Score: 0.2539
Training OCSVM with nu=0.005, gamma=0.0001
F1 Score: 0.2897
Training OCSVM with nu=0.005, gamma=0.001
F1 Score: 0.2850
Training OCSVM with nu=0.005, gamma=0.01
F1 Score: 0.2606
Training OCSVM with nu=0.01, gamma=scale
F1 Score: 0.2561
Training OCSVM with nu=0.01, gamma=auto
F1 Score: 0.2536
Training OCSVM with nu=0.01, gamma=0.0001
F1 Score: 0.2891
Training OCSVM with nu=0.01, gamma=0.001
F1 Score: 0.2842
Training OCSVM with nu=0.01, gamma=0.01
F1 Score: 0.2609
Training OCSVM with nu=0.02, gamma=scale
F1 Score: 0.2579
Training OCSVM with nu=0.02, gamma=auto
F1 Score: 0.2536
Training

In [9]:
print("Best Parameters:")
print(best_params)
print("Best F1 Score:", best_score)

Best Parameters:
{'nu': 0.001, 'gamma': 0.0001}
Best F1 Score: 0.29610236938647283


In [10]:
# Optional: Evaluate final model
from sklearn.metrics import classification_report, confusion_matrix
final_preds = best_model.predict(X_val_scaled)

print(confusion_matrix(y_val, final_preds))
print(classification_report(y_val, final_preds, target_names=["Attack", "Normal"]))

[[ 43827  10794]
 [197578 197720]]
              precision    recall  f1-score   support

      Attack       0.18      0.80      0.30     54621
      Normal       0.95      0.50      0.65    395298

    accuracy                           0.54    449919
   macro avg       0.56      0.65      0.48    449919
weighted avg       0.86      0.54      0.61    449919



In [20]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_true, preds))
print(classification_report(y_true, preds, target_names=["Attack", "Normal"]))


[[ 49483   5138]
 [286084 109214]]
              precision    recall  f1-score   support

      Attack       0.15      0.91      0.25     54621
      Normal       0.96      0.28      0.43    395298

    accuracy                           0.35    449919
   macro avg       0.55      0.59      0.34    449919
weighted avg       0.86      0.35      0.41    449919



In [11]:
import joblib

joblib.dump(best_model, 'ocsvm_model_best.pkl')
joblib.dump(scaler, 'scaler_best.pkl')

['scaler_best.pkl']